In [4]:
import tensorflow as tf


import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

import os

import numpy as np
import pandas as pd
from window_generator import WindowGenerator

from run_automation_utils import Params, init_model


In [5]:
def prep_window_generator(station_id):
    df = pd.read_feather(f"../../data/pollution/processed/interpolated/pm10/{station_id}.feather")
    df.set_index("timestamp", inplace=True)

    timestamp_s = df.index.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    year = (365.2425) * day

    df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
    df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
    df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
    df.reset_index(inplace=True)

    # Convert to radians.
    wd_rad = df.pop("winddirection_10m") * np.pi / 180
    df["winddirection_10m_sin"] = np.sin(wd_rad)
    df["winddirection_10m_cos"] = np.cos(wd_rad)

    return WindowGenerator(input_width=24 * 7, label_width=24, shift=24, df=df.copy(), label_columns=["pm10"])


In [6]:
# window_generator_814 = prep_window_generator(814)
window_generator_530 = prep_window_generator(530)
window_generator_538 = prep_window_generator(538)


In [9]:
from typing import Dict


run_params: Dict[str, Params] = {}
run_params["station_530_7lstmlayer"] = Params(
    window_generator_530, lstm_layer_count=7, lstm_sizes=[32, 32, 32, 32, 32, 32, 32]
)
run_params["station_538_7lstmlayer"] = Params(
    window_generator_538, lstm_layer_count=7, lstm_sizes=[32, 32, 32, 32, 32, 32, 32]
)


In [10]:
import logging

logger = logging.getLogger("run_logger")
fileHandler = logging.FileHandler("run.logs.txt")
logger.addHandler(fileHandler)


In [11]:
for run_id, params in run_params.items():
    logger.error("Start run: " + run_id)
    model = init_model(params)

    loss = tf.keras.losses.MeanSquaredError()
    if params.loss_function == "huber":
        loss = tf.keras.losses.huber
    model.compile(loss=loss, optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()])

    name = params.to_name(100)
    temp_name = name
    append_index = 0
    while os.path.exists("../models/" + temp_name):
        append_index += 1
        temp_name = name + "_" + str(append_index)
    if append_index != 0:
        name += "_" + str(append_index)

    run = neptune.init_run(
        project="data-mining-team2/model-tests",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4Mzg2ZWZmYi05YzRlLTQ3ODYtOWE1NC1mNDM4OTM1ZjNlOTkifQ==",
        custom_run_id=run_id,
        name=name,
        source_files=["./model_tests.ipynb"],
    )

    run["parameters"] = params.to_dict()

    neptune_cbk = NeptuneCallback(run=run, base_namespace="training")

    try:
        for epoch_count_factor in range(1, 5):
            epoch_count = 50 * epoch_count_factor
            history = model.fit(
                params.window_generator.train,
                epochs=50,
                validation_data=params.window_generator.val,
                callbacks=[neptune_cbk],
            )

            eval_metrics = model.evaluate(params.window_generator.test, verbose=0)
            for j, metric in enumerate(eval_metrics):
                run[f"eval/epoch_{epoch_count}/{model.metrics_names[j]}"] = metric

            name = params.to_name(epoch_count)
            temp_name = name
            append_index = 0
            while os.path.exists("../models/" + temp_name):
                append_index += 1
                temp_name = name + "_" + str(append_index)
            if append_index != 0:
                name += "_" + str(append_index)

            model.save("../models/" + name, include_optimizer=False)

            run["model_names/" + str(epoch_count) + "epochs"] = name
    except Exception as e:
        logger.error(e)
        pass
    run.stop()
    logger.error("Finished run: " + run_id)


e:\FHBielefeld\Master\DataMining\feinstaubprojekt-polen\.env\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/data-mining-team2/model-tests/e/MOD-3
Epoch 1/50


e:\FHBielefeld\Master\DataMining\feinstaubprojekt-polen\.env\lib\site-packages\neptune\common\warnings.py:62: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'window_generator.WindowGenerator'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  warnings.warn(


682/682 [==============================] - 12s 11ms/step - loss: 165.5907 - mean_absolute_error: 8.9962 - val_loss: 144.7167 - val_mean_absolute_error: 8.4626
Epoch 2/50
682/682 [==============================] - 7s 11ms/step - loss: 131.9719 - mean_absolute_error: 8.1322 - val_loss: 136.7180 - val_mean_absolute_error: 8.2638
Epoch 3/50
682/682 [==============================] - 7s 10ms/step - loss: 128.5361 - mean_absolute_error: 8.0262 - val_loss: 146.9202 - val_mean_absolute_error: 8.5511
Epoch 4/50
682/682 [==============================] - 7s 10ms/step - loss: 126.6649 - mean_absolute_error: 7.9563 - val_loss: 126.2209 - val_mean_absolute_error: 8.0017
Epoch 5/50
682/682 [==============================] - 7s 10ms/step - loss: 123.1667 - mean_absolute_error: 7.8335 - val_loss: 120.6623 - val_mean_absolute_error: 7.7327
Epoch 6/50
682/682 [==============================] - 7s 11ms/step - loss: 122.7792 - mean_absolute_error: 7.8160 - val_loss: 118.5955 - val_mean_absolute_error: 7.7

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_50_epochs_2lstm-layer_64,64_dropout-0-4_2\assets


...

Epoch 1/50
682/682 [==============================] - 7s 11ms/step - loss: 30.9118 - mean_absolute_error: 4.0646 - val_loss: 82.5763 - val_mean_absolute_error: 6.4367
Epoch 2/50
682/682 [==============================] - 8s 11ms/step - loss: 30.2372 - mean_absolute_error: 4.0240 - val_loss: 83.6962 - val_mean_absolute_error: 6.4437
Epoch 3/50
682/682 [==============================] - 8s 11ms/step - loss: 30.4515 - mean_absolute_error: 4.0340 - val_loss: 88.5433 - val_mean_absolute_error: 6.6128
Epoch 4/50
682/682 [==============================] - 7s 11ms/step - loss: 30.8432 - mean_absolute_error: 4.0567 - val_loss: 85.4962 - val_mean_absolute_error: 6.5323
Epoch 5/50
682/682 [==============================] - 7s 10ms/step - loss: 30.1685 - mean_absolute_error: 4.0110 - val_loss: 85.7541 - val_mean_absolute_error: 6.5265
Epoch 6/50
682/682 [==============================] - 7s 10ms/step - loss: 30.2256 - mean_absolute_error: 4.0169 - val_loss: 83.3753 - val_mean_absolute_error: 6.459

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_2lstm-layer_64,64_dropout-0-4_2\assets


INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_2lstm-layer_64,64_dropout-0-4_2\assets


Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/data-mining-team2/model-tests/e/MOD-3/metadata


ERROR:run_logger:Finished run: station_538_without_sin
